In [ ]:
import os

if os.name == 'nt':
    measuring_root = "D:/measuring"
else:
    measuring_root = "/Users/jesse/Thesis/Code"

execfile(os.path.join(measuring_root, "analysis/scripts/setup_analysis.py"))
import analysis.lib.purification.purify_delayfb as pu_delayfb;reload(pu_delayfb)
import analysis.lib.fastcarboncontrol.fcc as fcc; reload(fcc)
import matplotlib as mpl
mpl.style.use('seaborn-bright')
%matplotlib inline

In [ ]:
reload(pu_delayfb)
carbons = [2,4,5,3,6,7]

single_C_decay_data = dict()

for c in carbons:
    data = dict()
    data["x"], data["y"], data["y_u"], data["fr"] = \
    pu_delayfb.number_of_repetitions(contains = 'sweep_number_of_reps_C%d_X' % (c),do_fit = True,fixed=[0,2,5,6,4], ret_data_fit=True)
    single_C_decay_data[str(c)] = data

In [ ]:
reload(pu_delayfb)
carbons = [2,4,5,3,6,7]

repump_time_data = dict()

for c in carbons:
    data = dict()
    data["x"], data["y"], data["y_u"], data["fr"] = \
    pu_delayfb.average_repump_time(contains = 'Sweep_Repump_time_C%d_X' % (c),do_fit = True,fixed=[], ret_data_fit=True)
    repump_time_data[str(c)] = data

In [ ]:
import itertools
reload(pu_delayfb)

carbons = [2,4,5,3,6,7]
carbon_combis = list(itertools.combinations(carbons, 2))

print("All carbon combinations: " + str(carbon_combis))

carbon_combis = carbon_combis[:11]

print("Selected carbon combinations: " + str(carbon_combis))

C13_X_phase = 0.0

msmt_sweep_limits = [
    (10, 190, 30),
    (190, 340, 30),
    (340, 850, 120),
]

msmt_templates = [
    {
        'carbon_encoding':              'serial_swap',
        'carbon_swap_el_states':        ['Z', 'Z'],
        'Tomography_list': [
            # ['X', 'I'],
            # ['I', 'X'],
            ['X', 'X'],
            [C13_X_phase + 45.0, C13_X_phase + 45.0],
            [C13_X_phase + 45.0, C13_X_phase - 45.0],
        ]
    },
    {
        'carbon_encoding':              'MBE',
        'carbon_swap_el_states':        ['X'],
        'Tomography_list': [
            ['X', 'X'],
            ['Y', 'Y'],
            ['Z', 'Z'],
            ['X', 'Y'],
            ['Y', 'X'],
        ]
    },
    {
        'carbon_encoding':              'MBE',
        'carbon_swap_el_states':        ['-X'],
        'Tomography_list': [
            ['X', 'X'],
            ['Y', 'Y'],
            ['Z', 'Z'],
            ['X', 'Y'],
            ['Y', 'X'],
        ]
    }
]

results = {}

for combi in carbon_combis:
    combi_key = "".join([str(c) for c in combi])
    results[combi_key] = {}
    for m in msmt_templates:
        dictkey = "%s_%s" % (m['carbon_encoding'], "".join(m['carbon_swap_el_states']))
        results[combi_key][dictkey] = {}
        folder_names = []
        for d_i, d in enumerate(msmt_sweep_limits):
            name_template = "phase_fb_delayline_C%s_%s_%s_sec%d" % (
                    "".join([str(c) for c in combi]),
                    m['carbon_encoding'],
                    "".join(m['carbon_swap_el_states']),
                    d_i
            )
            folder_names += [name_template]
        print folder_names
        for t in ["".join([str(tbasis) for tbasis in tbases]) for tbases in m['Tomography_list']]:
            print("Tomo: %s" % t)
            data = dict()
            data["x"], data["y"], data["y_u"], data["fr"] = \
            pu_delayfb.number_of_repetitions_stitched(
                contains="",
                multi_contains=folder_names,
                do_fit=True,
                x_only=True,
                tomo_basis=t,
                fit_n=1.0,
                fixed=[0,2,5,6,4],
                ret_data_fit=True
            )
            results[combi_key][dictkey][t] = data
                
            
            
# import copy
# msmts_and_data = copy.copy(msmts)

# for m in msmts.iterkeys():
#     print("Msmt: "+m)
#     ots = msmts[m]["ots"]
#     msmts_and_data[m]["tomos_data"] = dict()
#     for t in msmts[m]["tomos"]:
#         print("Tomo: %s - %s" % (m, t))
#         data = dict()
#         data["x"], data["y"], data["y_u"], data["fr"] = \
#         pu_delayfb.number_of_repetitions_stitched(
#             contains="phase_fb_delayline",
#             older_thans=ots,
#             do_fit=True,
#             x_only=True,
#             tomo_basis=t,
#             fit_n=1.0,
#             fixed=[0,2,5,6,4],
#             ret_data_fit=True
#         )
#         msmts_and_data[m]["tomos_data"][t] = data

In [ ]:
interesting_msmts = [
    ("MBE_-X", "XX", r"MBE -X, $\langle XX \rangle$"), 
    ("MBE_-X", "ZZ", r"MBE -X, $\langle ZZ \rangle$"), 
    ("MBE_X", "XX", r"MBE +X, $\langle XX \rangle$"), 
    ("MBE_X", "ZZ", r"MBE +X, $\langle ZZ \rangle$"), 
    ("serial_swap_ZZ", "45.045.0", r"sep. XX, $\langle X+45\degree, X+45\degree \rangle$"),
    ("serial_swap_ZZ", "45.0-45.0", r"sep. XX, $\langle X+45\degree, X-45\degree \rangle$")
]

fit_xvals = np.linspace(0.,1000., 2000.)

colors = ['b','r','g','m','y','c','k']

for combi_key in results.iterkeys():
    fig = plt.figure()
    ax = plt.subplot()
    
    i = 0
    for i_m in interesting_msmts:
        data = results[combi_key][i_m[0]][i_m[1]]
        plt.errorbar(
            data["x"], data["y"], data["y_u"], 
            fmt='o', color=colors[i], #"C%d" % i, 
            label = r"%s, $N_{1/e} = %d \pm %d$" % (i_m[2], np.round(data["fr"]['params_dict']['T']),
                                                                     np.round(data["fr"]['error_dict']['T']))
        )
        plt.plot(fit_xvals, data["fr"]['fitfunc'](fit_xvals), color=colors[i]) #"C%d" % i)
        i += 1

    plt.ylim(-1,1)
    plt.xlim(0,1000)

    plt.xlabel("Number of LDE attempts")
    plt.ylabel(r"Expectation value")
    plt.title("Correlations for carbon combination C%s" % ", C".join(combi_key))

    lgd = plt.legend(loc=2, bbox_to_anchor=(1,1))    
    plt.savefig("batch-plots/carbon-correlations-%s.png" % combi_key, dpi=300, bbox_extra_artists=(lgd,), bbox_inches='tight')
    plt.show()
plt.close('all')

In [ ]:
fig = plt.figure()
ax = plt.subplot()
i=0
for c_str in single_C_decay_data.iterkeys():
    data = single_C_decay_data[c_str]
    plt.errorbar(
        data["x"], data["y"], data["y_u"], 
        fmt='o', color="C%d" % i, 
        label = r"C%s, $N_{1/e} = %d \pm %d$" % (c_str, np.round(data["fr"]['params_dict']['T']),
                                                                 np.round(data["fr"]['error_dict']['T']))
    )
    plt.plot(fit_xvals, data["fr"]['fitfunc'](fit_xvals), color="C%d" % i)
    i += 1

plt.ylim(0,1)
plt.xlim(0,600)

plt.xlabel("Number of LDE attempts")
plt.ylabel(r"Bloch vector length")
plt.title("Single-nuclear state decay")

lgd = plt.legend(loc=2, bbox_to_anchor=(1,1))    
plt.savefig("batch-plots/single-nuclear-state-decay.png", dpi=300, bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.show()
plt.close('all')